## **RandomForestClassifier**
Baseline Model

Last Update: 20240726

---

**I. Start Package Load**

Load Packages and authorize with Google Account Credentials

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import numpy as np
import pandas as pd
import pprint
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display
import re
import random

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
# from skmultilearn.model_selection import iterative_train_test_split
# from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.utils import indexable, _safe_indexing
from sklearn.utils.validation import _num_samples
from sklearn.model_selection._split import _validate_shuffle_split
from sklearn.multioutput import MultiOutputClassifier
from itertools import chain
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report, hamming_loss, f1_score, precision_score, recall_score

import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras.utils import plot_model
from keras.models import Model
from keras.metrics import BinaryAccuracy, F1Score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

Function to make dataframe for predicted to the target class

In [ ]:
def one_hot_to_labels(one_hot_array, class_labels):
  """
  Convert a one-hot encoded array to a list of class labels.

  Parameters:
  one_hot_array (numpy.ndarray): A 2D array where each row is a one-hot encoded vector.
  class_labels (list): A list of class labels corresponding to the one-hot encoded vectors.

  Returns:
  labels (list): A list of lists, where each inner list contains the class labels corresponding to the one-hot encoded vectors.

  """
  labels = []
  for row in one_hot_array:
    label_indices = np.where(row == 1)[0]
    row_labels = [class_labels[i] for i in label_indices]
    labels.append(row_labels)
  return labels

  # display classes
def k_hot_to_dense(k_hot_encoded):
    """
    Convert k-hot encoded vectors to dense vectors.

    Args:
        k_hot_encoded (numpy.ndarray): A 2D numpy array where each row is a k-hot encoded vector.

    Returns:
        list of lists: Each sublist contains the indices of the 1s for the corresponding k-hot encoded vector.
    """
    dense_vectors = []
    for k_hot in k_hot_encoded:
        dense_vector = np.where(k_hot == 1)[0].tolist()
        dense_vectors.append(dense_vector)
    return dense_vectors

def extract_class_names(values_list, indices_list):
    return [[[values_list[i]] for i in indices] for indices in indices_list]

**II. Read in Data**

In [5]:
# Get current working directory
cwd = os.getcwd()
print(cwd)

/content


In [6]:
# Load Class Labels
indices = np.load(os.path.join(base_path, 'normal_sampled_data/.npy'), allow_pickle=True)

NameError: name 'base_path' is not defined

In [7]:
# Read in Oversampled Data from Jason
base_path = '/Users/than/uc_berkeley/courses/207/final_project/data'

X_train_idx = np.load(os.path.join(base_path, 'oversampled_data/over_train_indices.npy'))
X_train_over = np.load(os.path.join(base_path, 'oversampled_data/over_X_train.npy'))
Y_train_year_over = np.load(os.path.join(base_path, 'oversampled_data/over_Y_train_year.npy'))
Y_train_make_over = np.load(os.path.join(base_path, 'oversampled_data/over_Y_train_make.npy'))
Y_train_model_over = np.load(os.path.join(base_path, 'oversampled_data/over_Y_train_model.npy'))
# Y_train_combined_over = np.load(os.path.join(base_path, 'oversampled_data/over_Y_train_combined'))

X_val_idx = np.load(os.path.join(base_path, 'oversampled_data/val_indices.npy'))
X_val = np.load(os.path.join(base_path, 'oversampled_data/X_val.npy'))
Y_val_year = np.load(os.path.join(base_path, 'oversampled_data/Y_val_year.npy'))
Y_val_make = np.load(os.path.join(base_path, 'oversampled_data/Y_val_make.npy'))
Y_val_model = np.load(os.path.join(base_path, 'oversampled_data/Y_val_model.npy'))
# Y_val_combined = np.load(os.path.join(base_path, 'oversampled_data/Y_val_combined'))

X_test_idx = np.load(os.path.join(base_path, 'oversampled_data/test_indices.npy'))
X_test = np.load(os.path.join(base_path, 'oversampled_data/X_test.npy'))
Y_test_year = np.load(os.path.join(base_path, 'oversampled_data/Y_test_year.npy'))
Y_test_make = np.load(os.path.join(base_path, 'oversampled_data/Y_test_make.npy'))
Y_test_model = np.load(os.path.join(base_path, 'oversampled_data/Y_test_model.npy'))
# Y_val_combined = np.load(os.path.join(base_path, 'oversampled_data/Y_test_combined'))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/than/uc_berkeley/courses/207/final_project/data/oversampled_data/over_train_indices.npy'

In [2]:
# Pull in Normal Sampled Data
X_train_idx = np.load(os.path.join(base_path, 'normal_sampled_data/train_indices.npy'))


NameError: name 'np' is not defined

In [ ]:
# Print Shape of Y_train_year_over and Y_test_year
print(Y_train_year_over.shape)
print(Y_val_year.shape)
print(Y_test_year.shape)

(7066, 93)
(503, 93)
(500, 93)


In [ ]:
# Bring in class labels
# grab class names for MLSMOTE, Confusion Matrix, and labeling post predictions
year_classes = np.load(os.path.join(base_path, 'year_col.npy'), allow_pickle=True)
year_classes = [str(i) for i in year_classes]
model_classes = np.load(os.path.join(base_path, 'model_col.npy'), allow_pickle=True)
make_classes = np.load(os.path.join(base_path, 'make_col.npy'), allow_pickle=True)
combined_classes = np.load(os.path.join(base_path, 'combined_col.npy'), allow_pickle=True)

---
#### **IV. Implement RandomForestClassifier for Make**


#### Start Classification for Make

Initalize the model

In [ ]:
# Start Random Forest Model for Make
make_model = MultiOutputClassifier(RandomForestClassifier(
    criterion='gini',
    n_estimators=385, # increasing this increases validation accuracy
    max_depth=None,
    min_samples_split=10, # increasing this to try to decrease overfitting
    min_samples_leaf=1, # adjusting this brings validation accuracy to 0
    random_state=42,
    n_jobs=-1
    ))

make_model.fit(X_train_over, Y_train_make_over)

MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=10,
                                                       n_estimators=385,
                                                       n_jobs=-1,
                                                       random_state=42))

In [ ]:
# Make predictions for test, train, and val
Y_train_make_pred = make_model.predict(X_train_over)
Y_val_make_pred = make_model.predict(X_val)
Y_test_make_pred = make_model.predict(X_test)

#### Classification Report for Make

Precision = TP / TP + FP

Recall = TP / TP + FN

Precision is really high so in most cases we are basically not predicting the correct class for a row that is not that class. (Basically, we aren't predicting Acura for a BMW class)

Recall is is typically really low because we may be not predicting Acura for cases where the part is ment for an Acura.

In [ ]:
# Print the accuracy scores for test, val and train
print(f"Accuracy for the test set: {accuracy_score(Y_test_make, Y_test_make_pred)}")
print(f"Accuracy for the validation set: {accuracy_score(Y_val_make, Y_val_make_pred)}")
print(f"Accuracy for the oversampled training set: {accuracy_score(Y_train_make_over, Y_train_make_pred)}")

Accuracy for the test set: 0.686
Accuracy for the validation set: 0.6739562624254473
Accuracy for the oversampled training set: 0.8720634022077555


In [ ]:
print(classification_report(Y_test_make, Y_test_make_pred, output_dict=False, target_names=make_classes, zero_division=np.nan))

                         precision    recall  f1-score   support

                     AC        nan       nan       nan         0
             AM General        nan      0.00      0.00         1
                    AMC       1.00      0.50      0.67         2
                  Acura       1.00      0.43      0.60        14
             Alfa Romeo        nan      0.00      0.00         1
      American LaFrance        nan       nan       nan         0
           Aston Martin        nan       nan       nan         0
                 Auburn        nan       nan       nan         0
                   Audi       0.93      0.87      0.90        15
                 Austin        nan       nan       nan         0
                    BMW       1.00      0.96      0.98        27
                Bentley        nan       nan       nan         0
              Blue Bird        nan       nan       nan         0
                  Buick       1.00      0.57      0.72        30
               Cadillac 

In [ ]:
# Print out the macro precision
print(f"Weighted Precision: {precision_score(Y_test_make, Y_test_make_pred, average='weighted', zero_division=np.nan)}")

Weighted Precision: 0.9987060041407868


In [ ]:
# Print Recall
print(f"Weighted Recall: {recall_score(Y_test_make, Y_test_make_pred, average='weighted', zero_division=np.nan)}")

Weighted Recall: 0.691764705882353


In [ ]:
# Print F-1 Score
print(f"Weighted F1 Score: {f1_score(Y_test_make, Y_test_make_pred, average='weighted', zero_division=np.nan)}")

Weighted F1 Score: 0.7967159454588449


Look into predicted values and the target in the test set

In [ ]:
# Compare the predictions to the target
y_pred_labels = one_hot_to_labels(Y_test_make_pred, make_classes)
y_true_labels = one_hot_to_labels(Y_test_make, make_classes)

In [ ]:
# Check year again
dense_vector = k_hot_to_dense(Y_test_make_pred)
dense_names = extract_class_names(make_classes, dense_vector)
dense_true_vector = k_hot_to_dense(Y_test_make)
true_names = extract_class_names(make_classes, dense_true_vector)
comparison_df = pd.DataFrame({'Index': X_test_idx, 'Predicted': dense_names, 'Target': true_names})

# Filter where Target is empty
comparison_df

,Index,Predicted,Target
0,2,[[Nissan]],[[Nissan]]
1,3,[[Ford]],[[Ford]]
2,8,[],[[International Harvester]]
3,22,"[[Buick], [Chevrolet], [GMC], [Hummer], [Isuzu...","[[Buick], [Chevrolet], [GMC], [Hummer], [Isuzu..."
4,34,[[Buick]],[[Buick]]
5,37,[[Volkswagen]],[[Volkswagen]]
6,60,[],[[Honda]]
7,70,[],"[[INFINITI], [Nissan]]"
8,71,"[[Buick], [Chevrolet]]","[[Buick], [Chevrolet]]"
9,90,[[Mitsubishi]],[[Mitsubishi]]


In [ ]:
# Save off predictions in numpy format for test, train and val
np.save(os.path.join(base_path, 'oversampled_data/output/Y_test_make_pred.npy'), Y_test_make_pred)
np.save(os.path.join(base_path, 'oversampled_data/output/Y_val_make_pred.npy'), Y_val_make_pred)
np.save(os.path.join(base_path, 'oversampled_data/output/Y_train_make_pred.npy'), Y_train_make_pred)

---

---
#### **V. Implement RandomForestClassifier for Year**


In [ ]:
# Start Random Forest Model for Make
year_model = MultiOutputClassifier(RandomForestClassifier(
    criterion='gini',
    n_estimators=385, # increasing this increases validation accuracy
    max_depth=None,
    min_samples_split=10, # increasing this to try to decrease overfitting
    min_samples_leaf=1, # adjusting this brings validation accuracy to 0
    random_state=42,
    n_jobs=-1
    ))

year_model.fit(X_train_over, Y_train_year_over)

MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=10,
                                                       n_estimators=385,
                                                       n_jobs=-1,
                                                       random_state=42))

In [ ]:
# Make predictions for test, train, and val
Y_train_year_pred = year_model.predict(X_train_over)
Y_val_year_pred = year_model.predict(X_val)
Y_test_year_pred = year_model.predict(X_test)

In [ ]:
# Print the accuracy scores for test, val and train
print(f"Accuracy for the test set: {accuracy_score(Y_test_year, Y_test_year_pred)}")
print(f"Accuracy for the validation set: {accuracy_score(Y_val_year, Y_val_year_pred)}")
print(f"Accuracy for the oversampled training set: {accuracy_score(Y_train_year_over, Y_train_year_pred)}")

Accuracy for the test set: 0.754
Accuracy for the validation set: 0.731610337972167
Accuracy for the oversampled training set: 0.8898952731389754


In [ ]:
print(classification_report(Y_test_year, Y_test_year_pred, output_dict=False, target_names=year_classes, zero_division=np.nan))

              precision    recall  f1-score   support

        1923        nan       nan       nan         0
        1924        nan       nan       nan         0
        1925        nan       nan       nan         0
        1926        nan       nan       nan         0
        1927        nan       nan       nan         0
        1928        nan       nan       nan         0
        1929        nan       nan       nan         0
        1930        nan       nan       nan         0
        1931        nan       nan       nan         0
        1932        nan       nan       nan         0
        1933        nan       nan       nan         0
        1934        nan       nan       nan         0
        1935        nan       nan       nan         0
        1942        nan       nan       nan         0
        1946        nan       nan       nan         0
        1947        nan       nan       nan         0
        1948        nan       nan       nan         0
        1949        nan    

In [ ]:
# Print Macro Precision
print(f"Macro Precision: {precision_score(Y_test_year, Y_test_year_pred, average='weighted', zero_division=np.nan)}")

Macro Precision: 0.9901664580201431


In [ ]:
# Print Recall
print(f"Recall: {recall_score(Y_test_year, Y_test_year_pred, average='weighted', zero_division=np.nan)}")

Recall: 0.8209085614443797


In [ ]:
# Print F-1 Score
print(f"F1 Score: {f1_score(Y_test_year, Y_test_year_pred, average='weighted', zero_division=np.nan)}")

F1 Score: 0.8964553877479561


In [ ]:
# Save off the year predictions in numpy format for test, train and val
np.save(os.path.join(base_path, 'oversampled_data/output/Y_test_year_pred.npy'), Y_test_year_pred)
np.save(os.path.join(base_path, 'oversampled_data/output/Y_val_year_pred.npy'), Y_val_year_pred)
np.save(os.path.join(base_path, 'oversampled_data/output/Y_train_year_pred.npy'), Y_train_year_pred)

In [ ]:
# Check year again
dense_vector = k_hot_to_dense(Y_test_year_pred)
dense_names = extract_class_names(year_classes, dense_vector)
dense_true_vector = k_hot_to_dense(Y_test_year)
true_names = extract_class_names(year_classes, dense_true_vector)
comparison_df = pd.DataFrame({'Index': X_test_idx, 'Predicted': dense_names, 'Target': true_names})

# Filter where Target is empty
comparison_df

,Index,Predicted,Target
0,2,[[2018]],[[2018]]
1,3,[[1983]],[[1983]]
2,8,[[1957]],[[1957]]
3,22,"[[2002], [2003], [2004], [2005], [2006], [2007...","[[2002], [2003], [2004], [2005], [2006], [2007..."
4,34,[[2018]],[[2018]]
5,37,"[[1998], [1999], [2000], [2001], [2002], [2003...","[[1998], [1999], [2000], [2001], [2002], [2003..."
6,60,[[1991]],"[[1988], [1989], [1990], [1991]]"
7,70,"[[2002], [2003], [2004], [2005], [2006], [2007...","[[2002], [2003], [2004], [2005], [2006], [2007..."
8,71,[[2015]],[[2015]]
9,90,"[[1997], [1998], [1999]]","[[1997], [1998], [1999]]"


----

#### Implement the Random Forest Model for Car Model

In [ ]:
# Start Random Forest Model for Model
model_model = MultiOutputClassifier(RandomForestClassifier(
    criterion='gini',
    n_estimators=385, # increasing this increases validation accuracy
    max_depth=None,
    min_samples_split=10, # increasing this to try to decrease overfitting
    min_samples_leaf=1, # adjusting this brings validation accuracy to 0
    random_state=42,
    n_jobs=-1
    ))

model_model.fit(X_train_over, Y_train_model_over)

MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=10,
                                                       n_estimators=385,
                                                       n_jobs=-1,
                                                       random_state=42))

In [ ]:
# Make predictions for test, train, and val
Y_train_model_pred = model_model.predict(X_train_over)
Y_val_model_pred = model_model.predict(X_val)
Y_test_model_pred = model_model.predict(X_test)

In [ ]:
# Print the accuracy scores for test, val and train
print(f"Accuracy for the test set: {accuracy_score(Y_test_model, Y_test_model_pred)}")
print(f"Accuracy for the validation set: {accuracy_score(Y_val_model, Y_val_model_pred)}")
print(f"Accuracy for the oversampled training set: {accuracy_score(Y_train_model_over, Y_train_model_pred)}")

Accuracy for the test set: 0.316
Accuracy for the validation set: 0.268389662027833
Accuracy for the oversampled training set: 0.6413812623832437


In [ ]:
# Print Precision, Recall, and F1 Score
print(classification_report(Y_test_model, Y_test_model_pred, output_dict=False, target_names=model_classes, zero_division=np.nan))

                                  precision    recall  f1-score   support

                             100       1.00      1.00      1.00         1
                            128i        nan       nan       nan         0
                            135i        nan       nan       nan         0
                            1500       1.00      1.00      1.00         5
                            190E        nan       nan       nan         0
                               2       1.00      1.00      1.00         1
                             200       1.00      0.60      0.75         5
                            2002        nan       nan       nan         0
                           200SX       1.00      1.00      1.00         1
                           220Sb        nan       nan       nan         0
                            228i        nan      0.00      0.00         1
                           240SX        nan      0.00      0.00         1
                            240Z     

In [ ]:
# Print Macro Precision
print(f"Macro Precision: {precision_score(Y_test_model, Y_test_model_pred, average='weighted', zero_division=np.nan)}")

Macro Precision: 1.0


In [ ]:
# Print Recall
print(f"Recall: {recall_score(Y_test_model, Y_test_model_pred, average='weighted', zero_division=np.nan)}")

Recall: 0.41510791366906474


In [ ]:
# Print F-1 Score
print(f"F1 Score: {f1_score(Y_test_model, Y_test_model_pred, average='weighted', zero_division=np.nan)}")

F1 Score: 0.5276667131667946


In [ ]:
# Check year again
dense_vector = k_hot_to_dense(Y_test_model_pred)
dense_names = extract_class_names(model_classes, dense_vector)
dense_true_vector = k_hot_to_dense(Y_test_model)
true_names = extract_class_names(model_classes, dense_true_vector)
comparison_df = pd.DataFrame({'Index': X_test_idx, 'Predicted': dense_names, 'Target': true_names})

# Filter where Target is empty
comparison_df

,Index,Predicted,Target
0,2,[[Altima]],[[Altima]]
1,3,[[Mustang]],[[Mustang]]
2,8,[[OtherModel]],[[OtherModel]]
3,22,"[[9-7x], [Ascender], [Bravada], [Canyon], [Col...","[[9-7x], [Ascender], [Bravada], [Canyon], [Col..."
4,34,[[Encore]],[[Encore]]
5,37,[],"[[Beetle], [Golf], [Jetta]]"
6,60,[],"[[CRX], [Civic]]"
7,70,[],"[[350Z], [Altima], [FX35], [G35], [I35], [M35]..."
8,71,[],"[[Encore], [Sonic], [Trax]]"
9,90,[],[[Montero]]


In [ ]:
np.save(os.path.join(base_path, 'oversampled_data/output/Y_test_model_pred.npy'), Y_test_model_pred)
np.save(os.path.join(base_path, 'oversampled_data/output/Y_val_model_pred.npy'), Y_val_model_pred)
np.save(os.path.join(base_path, 'oversampled_data/output/Y_train_model_pred.npy'), Y_train_model_pred)